In [82]:
import pandas as pd
import numpy as np

data = pd.read_csv("archive/anime.csv")
print(data.columns)

Index(['anime_id', 'title', 'type', 'score', 'scored_by', 'status', 'episodes',
       'start_date', 'end_date', 'source', 'members', 'favorites',
       'episode_duration', 'total_duration', 'rating', 'sfw', 'approved',
       'created_at', 'updated_at', 'start_year', 'start_season',
       'real_start_date', 'real_end_date', 'broadcast_day', 'broadcast_time',
       'genres', 'themes', 'demographics', 'studios', 'producers', 'licensors',
       'synopsis', 'background', 'main_picture', 'url', 'trailer_url',
       'title_english', 'title_japanese', 'title_synonyms'],
      dtype='object')


In [150]:
data['genres']

0        ['Action', 'Adventure', 'Drama', 'Fantasy']
1                 ['Action', 'Adventure', 'Fantasy']
2                                ['Action', 'Drama']
3                    ['Drama', 'Sci-Fi', 'Suspense']
4                                          ['Drama']
                            ...                     
24160                                             []
24161                          ['Action', 'Fantasy']
24162             ['Action', 'Adventure', 'Fantasy']
24163                                             []
24164                                             []
Name: genres, Length: 24165, dtype: object

## Odrzucenie kolumn

In [41]:
data_after = data
data_after = data_after.drop(["scored_by", 'members','created_at', 'updated_at','background', 'main_picture','url', 'trailer_url', 'title_japanese'], axis=1)


## Odrzucenie wierszy nie spełniających oczekiwanych kryteriów

In [42]:
selected_rows = data_after
selected_rows = selected_rows.loc[data_after['status'] == 'finished_airing']
selected_rows = selected_rows.loc[data_after['approved'] == True]

selected_rows

,anime_id,title,type,score,status,episodes,start_date,end_date,source,favorites,...,broadcast_time,genres,themes,demographics,studios,producers,licensors,synopsis,title_english,title_synonyms
0,5114,Fullmetal Alchemist: Brotherhood,tv,9.13,finished_airing,64.0,2009-04-05,2010-07-04,manga,204645,...,17:00:00,"['Action', 'Adventure', 'Drama', 'Fantasy']",['Military'],['Shounen'],['Bones'],"['Aniplex', 'Square Enix', 'Mainichi Broadcast...","['Funimation', 'Aniplex of America']",After a horrific alchemy experiment goes wrong...,Fullmetal Alchemist: Brotherhood,['Hagane no Renkinjutsushi: Fullmetal Alchemis...
1,11061,Hunter x Hunter (2011),tv,9.04,finished_airing,148.0,2011-10-02,2014-09-24,manga,185178,...,10:55:00,"['Action', 'Adventure', 'Fantasy']",[],['Shounen'],['Madhouse'],"['VAP', 'Nippon Television Network', 'Shueisha']",['VIZ Media'],Hunters devote themselves to accomplishing haz...,Hunter x Hunter,['HxH (2011)']
2,38524,Shingeki no Kyojin Season 3 Part 2,tv,9.07,finished_airing,10.0,2019-04-29,2019-07-01,manga,51931,...,00:10:00,"['Action', 'Drama']","['Gore', 'Military', 'Survival']",['Shounen'],['Wit Studio'],"['Production I.G', 'Dentsu', 'Mainichi Broadca...",['Funimation'],Seeking to restore humanity's diminishing hope...,Attack on Titan Season 3 Part 2,[]
3,9253,Steins;Gate,tv,9.08,finished_airing,24.0,2011-04-06,2011-09-14,visual_novel,173088,...,02:05:00,"['Drama', 'Sci-Fi', 'Suspense']","['Psychological', 'Time Travel']",[],['White Fox'],"['Frontier Works', 'Media Factory', 'Movic', '...",['Funimation'],Eccentric scientist Rintarou Okabe has a never...,Steins;Gate,[]
4,28851,Koe no Katachi,movie,8.95,finished_airing,1.0,2016-09-17,2016-09-17,manga,77431,...,NaN,['Drama'],['Romantic Subtext'],['Shounen'],['Kyoto Animation'],"['Shochiku', 'Pony Canyon', 'Kodansha', 'ABC A...","['Eleven Arts', 'NYAV Post']","As a wild youth, elementary school student Sho...",A Silent Voice,['The Shape of Voice']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,48189,Rocket Boy & Toro,tv,NaN,finished_airing,52.0,2008-01-01,2009-01-01,original,0,...,NaN,['Fantasy'],[],['Kids'],['Imagestone Inc.'],[],[],NaN,Rocket Boy & Toro,['Lokesboiwa Tolo']
20754,52466,AOKI & Taka no Tsume,ona,NaN,finished_airing,7.0,2014-10-22,2015-02-26,original,0,...,NaN,['Comedy'],[],[],[],[],[],A collaboration between Taka no Tsume and AOKI...,NaN,"['AOKI & Himitsukessha Taka no Tsume', 'AOKI &..."
20755,47237,Wo Yao Hui Jia,tv,NaN,finished_airing,26.0,2017-03-13,2017-01-01,NaN,0,...,NaN,"['Adventure', 'Fantasy']",[],['Kids'],[],[],[],NaN,NaN,[]
20757,52471,Lyeonglihan Neoguli,tv,NaN,finished_airing,63.0,1987-01-01,2009-01-01,original,0,...,NaN,[],[],['Kids'],[],[],[],Clever Raccoon Dog is a North Korean animated ...,The Little Bear,"['The Clever Racoon Dog', 'Yeongnihan Neoguri']"


## Konwersja danych

In [72]:
import datetime
from copy import deepcopy
def convert_to_seconds(time_str:str):
    if pd.isna(time_str):
        return time_str
    days = int(time_str.split(" ")[0])
    time_str = time_str[-8:]


    time_obj = datetime.datetime.strptime(time_str, r'%H:%M:%S')
    seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second
    seconds += days*24*60*60
    return seconds

def count_characters(text:str):
    if pd.isna(text):
        return 0
    return len(text)

def is_not_nan(field):
    return pd.isna(field)

In [73]:

converted_data =  deepcopy(selected_rows)
converted_data['episode_duration'] = converted_data['episode_duration'].apply(convert_to_seconds)
converted_data['total_duration'] = converted_data['total_duration'].apply(convert_to_seconds)

converted_data['synopsis_char_count'] = converted_data['synopsis'].apply(count_characters)
converted_data['title_char_count'] = converted_data['title'].apply(count_characters)
converted_data['start_date_day_of_year'] = pd.to_datetime(converted_data['start_date']).dt.dayofyear
converted_data['end_date_day_of_year'] = pd.to_datetime(converted_data['end_date']).dt.dayofyear
converted_data['broadcast_time'] = pd.to_timedelta(converted_data['broadcast_time']).dt.total_seconds() // 60


In [126]:
import ast
genres = converted_data['genres'].dropna()
genres = genres.apply(ast.literal_eval).explode().unique()
genres = list(genres)
genres.remove(np.nan)

add_name_genres = "is_"
for word in genres:
    converted_data[add_name_genres + word] = converted_data['genres'].apply(lambda x: word in x)


In [127]:

themes = converted_data['themes'].dropna()
themes = themes.apply(ast.literal_eval).explode().unique()
themes = list(themes)
themes.remove(np.nan)

add_name_themes = "is_"
for word in themes:
    converted_data[add_name_themes + word] = converted_data['themes'].apply(lambda x: word in x)

In [129]:
demographics = converted_data['demographics'].dropna()
demographics = demographics.apply(ast.literal_eval).explode().unique()
demographics = list(demographics)
demographics.remove(np.nan)

add_name_demographics = "is_"
for word in demographics:
    converted_data[add_name_demographics + word] = converted_data['demographics'].apply(lambda x: word in x)

## Przepisanie gotowych kolumn

In [145]:
ret = converted_data[['episodes', 'sfw', 'start_year','episode_duration','total_duration','synopsis_char_count','title_char_count','start_date_day_of_year','end_date_day_of_year','broadcast_time']]
for word in genres:
    ret = pd.concat([ret,converted_data[add_name_genres + word]], axis=1)

# for word in themes:
#     ret = pd.concat([ret,converted_data[add_name_themes + word]], axis=1)

for word in demographics:
    ret = pd.concat([ret,converted_data[add_name_demographics + word]], axis=1)

#klasy decyzyjne
ret = pd.concat([ret,converted_data['score']], axis=1)

ret = ret.dropna()

ret = ret.replace({True: 1, False: 0})

ret.to_csv("data/anime_data.csv")


In [146]:
# Tworzenie definicji dla Ludwig
input_features = []


for column in ret.columns:
    feature = {
        'name': column,
        'type': 'numerical' if pd.api.types.is_numeric_dtype(ret[column]) else 'category'
    }
    input_features.append(feature)

# Wyświetlanie definicji dla Ludwiga
print("input_features:")
for feature in input_features:
    print("  -")
    print("    name:", feature['name'])
    print("    type:", feature['type'])

input_features:
  -
    name: episodes
    type: numerical
  -
    name: sfw
    type: numerical
  -
    name: start_year
    type: numerical
  -
    name: episode_duration
    type: numerical
  -
    name: total_duration
    type: numerical
  -
    name: synopsis_char_count
    type: numerical
  -
    name: title_char_count
    type: numerical
  -
    name: start_date_day_of_year
    type: numerical
  -
    name: end_date_day_of_year
    type: numerical
  -
    name: broadcast_time
    type: numerical
  -
    name: is_Action
    type: numerical
  -
    name: is_Adventure
    type: numerical
  -
    name: is_Drama
    type: numerical
  -
    name: is_Fantasy
    type: numerical
  -
    name: is_Sci-Fi
    type: numerical
  -
    name: is_Suspense
    type: numerical
  -
    name: is_Supernatural
    type: numerical
  -
    name: is_Award Winning
    type: numerical
  -
    name: is_Romance
    type: numerical
  -
    name: is_Comedy
    type: numerical
  -
    name: is_Sports
    type: